In [4]:
import texfig

import tensorflow as tf
import numpy as np
import t3f
tf.set_random_seed(0)
np.random.seed(0)
import matplotlib.pyplot as plt
import metric_util as mt
import data_util as du
from nilearn import image
from tensorflow.python.util import nest
import copy
from nilearn import plotting
import mri_draw_utils as mrd
from scipy import optimize 
from nilearn.masking import compute_background_mask
from nilearn.masking import compute_epi_mask
from collections import OrderedDict
import pandas as pd
from scipy import stats
from nilearn.image import math_img
import cost_computation as cst
import tensor_util as tu
import nibabel as nib
import os
import metadata as mdt

In [5]:
class TuckerTensorCompletionRunner(object):
    
    def __init__(self, ground_truth_img, ground_truth, tensor_shape, x_init, mask_indices, z_scored_mask, 
                 sparse_observation_org,
                 norm_sparse_observation,
                 x_init_tcs,
                 ten_ones, max_tt_rank, observed_ratio, epsilon, train_epsilon, backtrack_const, logger, meta, d, z_score = 2):
        
        self.ground_truth_img = ground_truth_img
        self.ground_truth = ground_truth
        self.tensor_shape = tensor_shape
        self.x_init = x_init
        self.mask_indices = mask_indices
        self.z_scored_mask = z_scored_mask
        self.sparse_observation_org = sparse_observation_org
        self.norm_sparse_observation = norm_sparse_observation
        self.x_init_tcs = x_init_tcs
        self.ten_ones = ten_ones
        self.max_tt_rank = max_tt_rank
        self.observed_ratio = observed_ratio
        self.missing_ratio = 1.0 - observed_ratio
        self.logger = logger
        self.meta = meta
        self.d = d
        self.z_score = z_score
                 
        self.epsilon = epsilon
        self.train_epsilon = train_epsilon
        
        self.backtrack_const = backtrack_const
        
        self.init()
       
    def init(self):
        
        self.rse_cost_history = []
        self.train_cost_history = []
        self.tcs_cost_history = []
        self.tcs_z_scored_history = []
        self.cost_history = []
        self.scan_mr_folder = self.meta.create_scan_mr_folder(self.missing_ratio)
        self.scan_mr_iteration_folder = self.meta.create_scan_mr_folder_iteration(self.missing_ratio)
        self.images_mr_folder_iteration = self.meta.create_images_mr_folder_iteration(self.missing_ratio)
        self.suffix = self.meta.get_suffix(self.missing_ratio)
        
        self.logger.info(self.scan_mr_iteration_folder)
        self.logger.info(self.suffix)